# Roman-Cuts

A short tutorial on how to use `RomanCuts` to get cutouts from simulated images created with `RImTimSim` [Link](https://github.com/robertfwilson/rimtimsim).

In [27]:
import asdf
import pandas as pd
import lightkurve as lk
from astropy.io import fits
from astropy.table import Table
from astropy import units as u

In [3]:
FILTER = "F087"
SCA = 2
FIELD = 3

# Images

Let's inspect the image files:

### ASDF

This files have the image cutout cubes. In this case we will open the F087 filter, as it is the one of the smallest files (~130 MB). The file for the main filter F146 is ~6.5 GB, because it has ~6500 frames, but the file schema is the same.

In [ ]:
# get path for the ASDF files in disk
ff = f"dryrun_01/simulated_imgs/rimtimsim_WFI_lvl02_{FILTER}_SCA{SCA:02}_field{FIELD:02}_rampfitted_r1920c1920_256x256_sim.asdf"

# this one contains all the frames in the ASDF
cube = asdf.open(ff)
cube.info()

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 5.0.0
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata) ...
│   └─[1] (ExtensionMetadata) ...
└─roman (dict)
  ├─data (dict)
  │ ├─column (int): 1792
  │ ├─exposureno (NDArrayType) ...
  │ ├─flux (NDArrayType) ...
  │ ├─flux_err (NDArrayType) ...
  │ ├─quality (NDArrayType) ...
  │ ├─row (int): 1792
  │ └─time (NDArrayType) ...
  ├─meta (dict)
  │ ├─CREATOR (str): TRExS-roman-cuts
  │ └─17 not shown
  └─wcs (list) ...
Some nodes not shown.


In [12]:
# metadata
cube["roman"]["meta"]

{'CREATOR': 'TRExS-roman-cuts',
 'DEC_CEN': -29.20603569593083,
 'DETECTOR': 'SCA02',
 'DITHERED': False,
 'EQUINOX': 2000.0,
 'EXPOSURE': 54.72,
 'FIELD': 3,
 'FILTER': 'F087',
 'IMGSIZE': [256, 256],
 'MISSION': 'Roman-Sim',
 'NTIMES': 132,
 'RADESYS': 'FK5',
 'RA_CEN': 268.49141932159233,
 'READMODE': 'rampfitted',
 'SOFTWARE': 'rimtimsim_v2.0',
 'TELESCOP': 'Roman',
 'TEND': 2461520.3880308005,
 'TSTART': 2461450.751698133}

In [10]:
# flux image
print(cube["roman"]["data"]["flux"].shape)
# flux errors
print(cube["roman"]["data"]["flux_err"].shape)

(132, 256, 256)
(132, 256, 256)


In [ ]:
# WCSs for every exposure
cube["roman"]["wcs"][:3]


[WCS Keywords
 
 Number of WCS axes: 2
 CTYPE : 'RA---TAN' 'DEC--TAN' 
 CRVAL : 268.49716267365 -29.204747460217 
 CRPIX : 2044.0 2044.0 
 PC1_1 PC1_2  : 2.6187949455532e-05 1.457483151028e-05 
 PC2_1 PC2_2  : -1.5281148068944e-05 2.544301041483e-05 
 CDELT : 1.0 1.0 
 NAXIS : 0  0,
 WCS Keywords
 
 Number of WCS axes: 2
 CTYPE : 'RA---TAN' 'DEC--TAN' 
 CRVAL : 268.49715884725 -29.2048144657 
 CRPIX : 2044.0 2044.0 
 PC1_1 PC1_2  : 2.6187949502283e-05 1.457483143244e-05 
 PC2_1 PC2_2  : -1.5281147988825e-05 2.5443010459421e-05 
 CDELT : 1.0 1.0 
 NAXIS : 0  0,
 WCS Keywords
 
 Number of WCS axes: 2
 CTYPE : 'RA---TAN' 'DEC--TAN' 
 CRVAL : 268.49719453236 -29.204760419092 
 CRPIX : 2044.0 2044.0 
 PC1_1 PC1_2  : 2.6187949464574e-05 1.4574831495225e-05 
 PC2_1 PC2_2  : -1.5281148053448e-05 2.5443010423455e-05 
 CDELT : 1.0 1.0 
 NAXIS : 0  0]

### FITS

The FITS files have the full frame image (4088x4088 pixels).

In [16]:
ff = f"dryrun_01/simulated_imgs/rimtimsim_WFI_lvl02_{FILTER}_SCA{SCA:02}_field{FIELD:02}_rampfitted_exposureno_000000203_sim.fits"
hdul = fits.open(ff)
hdul.info()

Filename: dryrun_01/simulated_imgs/rimtimsim_WFI_lvl02_F087_SCA02_field03_rampfitted_exposureno_000000203_sim.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      38   (4088, 4088)   float64   
  1                1 ImageHDU         7   (4088, 4088)   float64   


In [17]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4088                                                  
NAXIS2  =                 4088                                                  
EXTEND  =                    T                                                  
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =               2044.0 / Pixel coordinate of reference point            
CRPIX2  =               2044.0 / Pixel coordinate of reference point            
PC1_1   =  2.6187949455532E-05 / Coordinate transformation matrix element       
PC1_2   =   1.457483151028E-05 / Coordinate transformation matrix element       
PC2_1   = -1.5281148068944E-05 / Coordinate transformation matrix element       
PC2_2   =   2.544301041483E-

In [18]:
# flux image
print(hdul[0].data.shape)
# flux errors
print(hdul[1].data.shape)

(4088, 4088)
(4088, 4088)


# Catalogs

First we take a look at th master input catalog for the image cutout. This table has the source ids `sicbro_id`, object sky and CCD positions, filter magnitudes (and fluxes), and a flag for the source type (EB, transit, variable, noise).

In [ ]:
catalog = pd.read_csv(
    "dryrun_01/catalogs/TRExS_dryrun_01_MASTER_input_catalog_v1.1_cutout.csv",
    index_col=0,
)
catalog

,sicbro_id,RA_DEG,DEC_DEG,MEAN_XCOL,MEAN_YCOL,F062,F087,F106,F129,F158,...,lowmassEB,lowRedNoise,hiRedNoise,variable,F087_flux,F087_flux_err,F146_flux,F146_flux_err,F213_flux,F213_flux_err
0,412,268.486137,-29.207724,1815.633710,1791.258215,17.6942,17.1567,16.8910,16.6252,16.4545,...,0,0,0,0,14215.648228,119.229393,24066.152710,155.132694,20372.609847,142.732652
1,485,268.492709,-29.205453,1942.561096,1956.774163,21.3311,20.1337,19.6439,19.1540,18.9670,...,0,0,0,0,916.150173,30.267973,2407.945583,49.070822,2204.381046,46.950836
2,680,268.488748,-29.206026,1853.014050,1880.456923,25.2901,23.5623,22.7456,21.9289,21.7775,...,0,0,0,0,38.951723,6.241132,186.297346,13.649079,179.641498,13.403041
3,953,268.494405,-29.203250,1948.810613,2047.119672,25.3561,25.2320,25.3867,25.5415,25.9286,...,0,0,0,0,8.368482,2.892833,4.745988,2.178529,3.132180,1.769797
4,992,268.485481,-29.206998,1787.326885,1802.807431,23.4039,21.9230,21.3025,20.6821,20.5238,...,0,1,0,1,176.297709,13.277715,587.444265,24.237250,559.353897,23.650664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20853,4972506,268.490400,-29.209570,1952.399376,1800.841806,20.0674,19.3571,19.0990,18.8409,18.6984,...,0,0,0,0,1873.297132,43.281603,3112.050732,55.785757,2721.740202,52.170300
20854,4972710,268.491005,-29.206618,1919.117790,1896.877473,19.9720,19.1784,18.8989,18.6195,18.4194,...,0,0,0,0,2208.445408,46.994100,3952.270567,62.867087,3577.063651,59.808558
20855,4972847,268.490970,-29.202617,1852.631122,2014.215558,18.2691,17.5345,17.2334,16.9322,16.7241,...,0,0,0,0,10037.987907,100.189759,18748.506089,136.925184,16906.225918,130.023944
20856,4972850,268.489604,-29.204565,1850.443519,1936.324893,20.1870,19.4660,19.2523,19.0385,18.9303,...,0,0,0,0,1694.519879,41.164546,2545.227859,50.450251,2197.084034,46.873063


In [24]:
catalog.columns

Index(['sicbro_id', 'RA_DEG', 'DEC_DEG', 'MEAN_XCOL', 'MEAN_YCOL', 'F062',
       'F087', 'F106', 'F129', 'F158', 'F184', 'F213', 'F146', 'transitHost',
       'dimEB', 'blendedEB', 'lowmassEB', 'lowRedNoise', 'hiRedNoise',
       'variable', 'F087_flux', 'F087_flux_err', 'F146_flux', 'F146_flux_err',
       'F213_flux', 'F213_flux_err'],
      dtype='object')

The next catalog is has the physical parameters of the objects injected in the simulation. Here we can find stellar parameters (e.g. `'star_mass', 'star_radius', 'star_teff', 'star_logg', 'star_met'`) and also planet parameters (e.g. `'transit_period', 'transit_t0_BRJD', 'planet_radius', 'planet_sma', 'planet_impact', 'planet_teff_eq'` etc). The `star_id` in the transit catalog correspond to the `sicbro_id` column in the master catalog. 

In [22]:
transits = pd.read_csv(
    "dryrun_01/catalogs/TRExS_dryrun_01_catalog_planet_sample_injected_parameters_v1.2_cutout.csv",
    index_col=0,
)
transits


,star_id,planet_id,transit_period,transit_t0_BRJD,transit_rp_rstar,transit_a_rstar,transit_inc,transit_ecc,transit_w,transit_limb1_F146,...,star_dist,star_Av,F146,F062,F087,F106,F129,F158,F184,F213
0,992,992.01,22.309497,109.159840,0.087227,89.272319,89.6958,0.2365,290.6,-0.0011,...,1.4718,0.553770,20.6162,23.4039,21.9230,21.3025,20.6821,20.5238,20.5966,20.6694
1,1162,1162.01,12.877802,116.463633,0.122008,74.741440,89.9823,0.1928,234.5,-0.0011,...,1.5375,0.515217,21.2190,24.0901,22.5772,21.9301,21.2831,21.1294,21.1996,21.2699
2,1402,1402.01,1.860683,105.828850,0.114076,16.326604,88.0672,0.0203,106.9,-0.0011,...,1.6271,0.460015,20.2955,22.8722,21.5010,20.9300,20.3590,20.1942,20.2826,20.3711
3,1402,1402.02,18.981238,118.055001,0.131633,76.794593,89.8190,0.0453,233.1,-0.0011,...,1.6271,0.460015,20.2955,22.8722,21.5010,20.9300,20.3590,20.1942,20.2826,20.3711
4,2164,2164.01,19.975283,121.095768,0.101033,78.411338,89.4225,0.1161,240.7,-0.0011,...,1.8336,0.629125,20.6939,23.3617,21.9522,21.3550,20.7578,20.6040,20.6749,20.7458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,4925691,4925691.01,309.334329,126.016327,0.098035,167.371082,89.8298,0.0729,239.5,0.0522,...,7.8232,2.313218,19.3714,20.4981,19.7327,19.5422,19.3517,19.3016,19.4260,19.5503
333,4927044,4927044.01,19.665658,123.144678,0.043836,26.375267,88.8434,0.0100,112.7,0.0522,...,8.4215,1.782229,19.3341,20.1779,19.5663,19.4232,19.2801,19.2751,19.4176,19.5601
334,4927560,4927560.01,6.959682,109.164397,0.071092,11.845354,88.5438,0.1137,223.4,0.0893,...,9.7611,1.854079,19.4152,20.2869,19.6579,19.5101,19.3623,19.3572,19.4971,19.6371
335,4927629,4927629.01,19.426635,110.167596,0.097059,23.908098,88.4913,0.0792,115.7,0.0893,...,8.1637,2.183099,19.0095,20.0559,19.3319,19.1548,18.9777,18.9451,19.0736,19.2021


In [23]:
transits.columns

Index(['star_id', 'planet_id', 'transit_period', 'transit_t0_BRJD',
       'transit_rp_rstar', 'transit_a_rstar', 'transit_inc', 'transit_ecc',
       'transit_w', 'transit_limb1_F146', 'transit_limb2_F146',
       'transit_limb1_F087', 'transit_limb2_F087', 'transit_limb1_F213',
       'transit_limb2_F213', 'transit_fp_fstar_F146', 'transit_fp_fstar_F087',
       'transit_fp_fstar_F213', 'transit_t0_sec', 'planet_n_transits',
       'planet_est_transit_snr', 'planet_radius', 'planet_sma',
       'planet_impact', 'planet_teff_eq', 'star_n_planets', 'star_mass',
       'star_radius', 'star_teff', 'star_logg', 'star_met', 'star_Mbol',
       'star_dist', 'star_Av', 'F146', 'F062', 'F087', 'F106', 'F129', 'F158',
       'F184', 'F213'],
      dtype='object')

# Light Curves

We have two types of light curves **pivot** and **extracted**. Pivot light curves are simulated time series with added noise, while extracted light curves where created by our PSF photometry pipeline from the image simulations.

### Extracted Light Curve

In [59]:
ff = "dryrun_01/lcs_extracted/roman_wfi_04579395_F146_dryrun01_lc.fits"

# we can create a lightkurve object
lc = lk.LightCurve.read(ff, time_column="TIME", time_format="jd", time_scale="tdb")
lc

time,FLUX,FLUX_ERR,CADENCENO,QUALITY
,electron / s,electron / s,,
Time,float32,float32,int32,int32
2461450.500301375,629.8412475585938,7.495388031005859,3,0
2461450.5104192654,612.6762084960938,7.418801784515381,11,0
2461450.52046775,598.5546264648438,7.272757530212402,19,0
2461450.5304623423,580.5967407226562,7.351845741271973,27,0
2461450.540556362,552.4468994140625,6.919987678527832,35,0
2461450.5506847147,589.47900390625,7.256985187530518,43,0
2461450.5607966566,573.531005859375,7.291995048522949,51,0
2461450.570822035,612.1283569335938,7.310596466064453,59,0


In [60]:
# or open the fits file
hdul = fits.open(ff)
hdul.info()

Filename: dryrun_01/lcs_extracted/roman_wfi_04579395_F146_dryrun01_lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      34   ()      
  1  LIGHTCURVE    1 BinTableHDU     24   6595R x 5C   [D, E, E, J, J]   


In [61]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standards                     
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T / file contains extensions                       
NEXTEND =                    2 / number of standard extensions                  
EXTNAME = 'PRIMARY '           / name of extension                              
EXTVER  =                    1 / extension version number (not format version)  
ORIGIN  = 'Unofficial data product' / organization resposible for file          
DATE    = '2025-08-13'         / file creation date.                            
CREATOR = 'TRExS-roman-cuts'   / pipeline job and program used t                
TELESCOP= 'Roman   '           / telescope                                      
INSTRUME= 'WFI     '           / detector type                                  
OBJECT  =  / string version 

### Pivot Light Curve

In [56]:
ff = "dryrun_01/lcs_pivot/TRExS_232842_F146_simlc.fits"

# we can create a lightkurve object
lc = lk.LightCurve.read(ff, time_column="TIME", time_format="jd", time_scale="tdb")
lc


time,FLUX,FLUX_ERR,CADENCENO,QUALITY
,electron / s,electron / s,,
Time,float32,float32,int32,int32
2461450.4999847086,912.5499877929688,6.239999771118164,3,0
2461450.510102599,916.0599975585938,6.239999771118164,11,0
2461450.520151083,910.0999755859375,6.239999771118164,19,0
2461450.530145676,910.0700073242188,6.239999771118164,27,0
2461450.540239695,919.5700073242188,6.239999771118164,35,0
2461450.550368048,901.969970703125,6.239999771118164,43,0
2461450.56047999,913.2999877929688,6.239999771118164,51,0
2461450.5705053685,919.4000244140625,6.239999771118164,59,0


In [57]:
# or open the fits file
hdul = fits.open(ff)
hdul.info()


Filename: dryrun_01/lcs_pivot/TRExS_232842_F146_simlc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      26   ()      
  1  LIGHTCURVE    1 BinTableHDU     24   6595R x 5C   [D, E, E, J, J]   


In [58]:
hdul[0].header


SIMPLE  =                    T / conforms to FITS standards                     
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T / file contains extensions                       
NEXTEND =                    2 / number of standard extensions                  
EXTNAME = 'PRIMARY '           / name of extension                              
EXTVER  =                    1 / extension version number (not format version)  
ORIGIN  = 'Unofficial data product' / organization resposible for file          
DATE    = '2025-07-08'         / file creation date.                            
CREATOR = 'trexs-roman-lcs.to_fits()' / pipeline job and program used t         
TELESCOP= 'ROMAN   '           / telescope                                      
INSTRUME= 'WFI     '           / detector type                                  
OBJECT  = '232842  '        